Verifique como o seu sistema armazena números de ponto flutuante em endereços adjacentes de memória, [*big-endian* ou *little-endian*](https://en.wikipedia.org/wiki/Endianness)?

Note que isso não é necessário no sistema do **Colab** que sabemos ser *big-endian*.

In [1]:
import numpy as np
# Para testar seu sistema para big-endian ou little-endian
import struct
print("big-endian   ", struct.pack(">f", 3.5).hex())
print("little-endian", struct.pack("<f", 3.5).hex())
print("qual deles foi igual a '0x40600000'?")

big-endian    40600000
little-endian 00006040
qual deles foi igual a '0x40600000'?


# **Explorando formato IEEE-754**

Como representamos números de ponto flutuante em computadores? A linguagem Python possui um módulo `struct` que permite com que comecemos a investigação.

In [4]:
import struct

value = np.pi
value_bytes = struct.pack(">f", value)
print("Em bytes:", value_bytes, "\nEm hexadecimal:", value_bytes.hex())

Em bytes: b'@I\x0f\xdb' 
Em hexadecimal: 40490fdb


Criamos uma função para converter do tipo nativo `bytes` ([documentação](https://docs.python.org/3/library/stdtypes.html#binaryseq)) para um número em binário:

In [5]:
def bytes_to_bin(bytes_value):
    hex_value = bytes_value.hex()
    ans = ''
    for n in hex_value:
        tmp = bin(int(n, base=16))[2:]
        if len(tmp) < 4:
            tmp = '0'*(4 - len(tmp)) + tmp
        ans += tmp
    return ans
  
value_bin = bytes_to_bin(value_bytes)
print("Em binário:", value_bin)

Em binário: 01000000010010010000111111011011


Podemos criar também a função inversa do mesmo:

In [6]:
def bin_to_bytes(bin_value):
    bin_value = ''.join(c for c in bin_value if c.isnumeric())
    ans = ''
    for i in range(0, len(bin_value), 4):
        ans += hex(int(bin_value[i:i+4], base=2))[2:]
    return bytes.fromhex(ans)
    
value_bytes = bin_to_bytes(value_bin)
print("Em bytes:", value_bytes, "\nEm hexadecimal:", value_bytes.hex())

Em bytes: b'@I\x0f\xdb' 
Em hexadecimal: 40490fdb


Nossa investigação pode ser mais completa ao gerarmos uma função que nos traz as informações necessárias:

In [ ]:
def bytes_to_float_info(bytes_value):

    def value_mantissa(mantissa):
        value = 0
        for i, b in enumerate(mantissa):
            value += 0 if b == '0' else 2**-(i+1)
        return value

    bin_value = bytes_to_bin(bytes_value) # valor em sequência de 0s e de 1s
    lb = len(bin_value) # 32 ou 64 bits
    sign = bin_value[0]
    exponent = bin_value[1:9] if lb == 32 else bin_value[1:12]
    mantissa = bin_value[9:] if lb == 32 else bin_value[12:]
    bias = 127 if lb == 32 else 1023 # 2**(len(exponent)-1) - 1
    s = 1 if sign == '0' else -1
    e = int(exponent, base=2) - bias
    m = value_mantissa(mantissa)
    value = s*(2**e)*(1 + m)
    return { 
        'sign': {'bin': sign, 'use as': '+' if s > 0 else '-'}, 
        'exponent': {'bin': exponent, 'raw': e + bias, 'use as': e}, 
        'mantissa': {'bin': mantissa, 'raw': m, 'use as': 1 + m},
        'bias': bias, 
        'nr_bits': lb,
        'value': value,
        'value in memory': f"{value:.100g}",
    }

num_info = bytes_to_float_info(value_bytes)

from pprint import pprint
pprint(num_info)

{'bias': 127,
 'exponent': {'bin': '10000000', 'raw': 128, 'use as': 1},
 'mantissa': {'bin': '11000000000000000000000', 'raw': 0.75, 'use as': 1.75},
 'nr_bits': 32,
 'sign': {'bin': '1', 'use as': '-'},
 'value': -3.5,
 'value in memory': '-3.5'}


A norma **IEEE-754** é que define o modo de armazenar um número de ponto flutuante na memória de um computador. A fórmula para o cálculo a partir dos bits armazenados chegarmos no número representado:

`npf = (-1)^s * 2^(e - bias) * (1 + m)`

onde `s` é o primeiro bit (0 = positivo, 1 = negativo); `e` é o expoente, a sequência de bits convertida de binário para decimal com algoritmo padrão; e `m` é a fração ou mantissa, a sequência final de bits convertida do bit mais significativo para o menos significativo com potências negativas de 2 (2^-1, 2^-2, 2^-3, etc...) a formar um número real menor do que 1. 

Para referência:

- *Precisão, nº bits, nº bits expoente, bias do expoente, nº bits mantissa*
- Simples, 32 bits, 8, 127, 23
- Dupla, 64 bits, 11, 1023, 52

**Assista à vídeo aula para maiores detalhes.**

Colocando tudo junto:

In [ ]:
def info_float(value, double_precision=False):
    endian = ">" # ">" big-endian, "<" little-endian (quase todos os sistemas, padrão é big-endian)
    value_format = endian + ("d" if double_precision else "f")
    value_bytes = struct.pack(value_format, value)
    return bytes_to_float_info(value_bytes)

Podemos agora experimentar com números distintos:

In [ ]:
real_number = 1/3 # <== altere aqui seu número em ponto flutuante

print("[Precisão simples]")
pprint(info_float(real_number))

print("\n[Precisão dupla]")
pprint(info_float(real_number, double_precision=True))

[Precisão simples]
{'bias': 127,
 'exponent': {'bin': '01111101', 'raw': 125, 'use as': -2},
 'mantissa': {'bin': '01010101010101010101011',
              'raw': 0.3333333730697632,
              'use as': 1.3333333730697632},
 'nr_bits': 32,
 'sign': {'bin': '0', 'use as': '+'},
 'value': 0.3333333432674408,
 'value in memory': '0.3333333432674407958984375'}

[Precisão dupla]
{'bias': 1023,
 'exponent': {'bin': '01111111101', 'raw': 1021, 'use as': -2},
 'mantissa': {'bin': '0101010101010101010101010101010101010101010101010101',
              'raw': 0.33333333333333326,
              'use as': 1.3333333333333333},
 'nr_bits': 64,
 'sign': {'bin': '0', 'use as': '+'},
 'value': 0.3333333333333333,
 'value in memory': '0.333333333333333314829616256247390992939472198486328125'}


Perceba que os números armazenados na memória **nem sempre são o que esperaríamos que fossem**!

## Diferença mínima entre números

Um pequeno experimento e podemos entender a necessidade e o uso de arredondamentos na conversão binário para ponto flutuante:

In [ ]:
number = 1/3 # <== altere aqui seu número em ponto flutuante
number_info = info_float(number)

bin_number = number_info['sign']['bin'] + number_info['exponent']['bin'] + number_info['mantissa']['bin']
print('orig', bin_number)

bin_number_alt = bin_number[:-1] + ('0' if bin_number[-1] == '1' else '1')
print('alt ', bin_number_alt)

number_alt_info = bytes_to_float_info(bin_to_bytes(bin_number_alt))

print('orig', number_info['value in memory'])
print('alt ', number_alt_info['value in memory'])


orig 00111110101010101010101010101011
alt  00111110101010101010101010101010
orig 0.3333333432674407958984375
alt  0.333333313465118408203125


Podemos ver que a diferença de um único bit menos significativo na mantissa leva a números diferentes no conjunto dos reais, e são números com uma distância significativa. 

No exemplo com `number = 1/3`, a distância entre eles está na ordem de 10^-8 e que é impossível para um computador representar qualquer número intermediário (por exemplo, `0.33333333`).

Vale notar que a **precisão da representação não possui relação alguma com o número de casas decimais dos números representados**.

In [ ]:
b = '01000000011000000000000000100000'
b = '0100000000001100000000000000010001000000000000000000000000000000'
print(len(b))
print('0'*32)
bytes_to_float_info(bin_to_bytes(b))

64
00000000000000000000000000000000


{'bias': 1023,
 'exponent': {'bin': '10000000000', 'raw': 1024, 'use as': 1},
 'mantissa': {'bin': '1100000000000000010001000000000000000000000000000000',
  'raw': 0.7500040531158447,
  'use as': 1.7500040531158447},
 'nr_bits': 64,
 'sign': {'bin': '0', 'use as': '+'},
 'value': 3.5000081062316895,
 'value in memory': '3.500008106231689453125'}

## Épsilon da máquina

Em aritmética de ponto flutuante, denomina-se épsilon da máquina o menor número que somado a 1 produza resultado diferente de 1, ou seja, que não é arredondado. O épsilon de máquina representa a exatidão relativa da aritmética do computador, e a sua existência é uma consequência direta da precisão finita da aritmética de ponto flutuante. O valor também é chamado de unidade de arredondamento ou menor número representável, e é simbolizado pela letra grega épsilon $\epsilon$.

Fonte: [Wikipedia](https://pt.wikipedia.org/wiki/Épsilon_de_máquina)

In [ ]:
machine_epsilon = 1.0
while 1.0 + machine_epsilon != 1.0:
  machine_epsilon /= 2.0

print(f"[precisão dupla] ϵ = {machine_epsilon}")

[precisão dupla] ϵ = 1.1102230246251565e-16


É realidade que 1 + ϵ = 1 ?? Podemos verificar:

In [ ]:
1 + machine_epsilon == 1

True

## Exemplo de Cálculo com Erro Numérico

Qual o seno de π?


In [ ]:
import numpy as np

print(np.sin(np.pi))

1.2246467991473532e-16


![Círculo Unitário, trigonometria](https://i.pinimg.com/736x/99/22/d6/9922d68e6db185d8c86a34161ba5d68f--unit-circle-trigonometry-physics-classroom.jpg)

Logo, **sen(π) = 0**, não `1.2246467991473532e-16`!

Mas, π = 3.141592653589793238462643383279502884197169399375105820974944592307816406286...

Veja mais em: [100mil dígitos de π](http://www.geom.uiuc.edu/~huberty/math5337/groupe/digits.html)

Para um computador:

In [ ]:
info_pi = info_float(np.pi, double_precision=True)
pprint(info_pi)

{'bias': 1023,
 'exponent': {'bin': '10000000000', 'raw': 1024, 'use as': 1},
 'mantissa': {'bin': '1001001000011111101101010100010001000010110100011000',
              'raw': 0.5707963267948966,
              'use as': 1.5707963267948966},
 'nr_bits': 64,
 'sign': {'bin': '0', 'use as': '+'},
 'value': 3.141592653589793,
 'value in memory': '3.141592653589793115997963468544185161590576171875'}


In [ ]:
print('π na memória (double) =', info_pi['value in memory'])
print('π na realidade        =','3.141592653589793238462643383279502884197169399375...')
print('                                         ^')

π na memória (double) = 3.141592653589793115997963468544185161590576171875
π na realidade        = 3.141592653589793238462643383279502884197169399375...
                                         ^


Veja que existe diferença a partir da 16ª casa decimal (erro na ordem de 10^-16), ou seja, π não é completamente representável por um número de bits fixo, seja feita em qualquer precisão em um computador. Para ser honesto, nem mesmo pelo sistema decimal (ver [números transcendentais](https://pt.wikipedia.org/wiki/Número_transcendente)).

Perceba que `sen(3.141592653589793115997963468544185161590576171875)` é, de fato, algo próximo de `1.22e-16`. Mas também perceba que `1.22e-16` está na ordem do *épsilon de máquina* e, portanto, **sendo coerentes podemos assumir como zero nas nossas análises**.

Logo, diferenças ínfimas na precisão de representação de um número podem levar a discrepâncias no resultado esperado de qualquer cálculo numérico, em especial com algoritmos que utilizam inúmeras operações aritméticas. É necessário que sempre estejamos atentos a isso.